In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np 
import os 


In [10]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['log_device_placement'] = 'true'

## Implement a custom layer that performs Layer Normalization

In [11]:
class myNormalizingLayer(keras.layers.Layer):
    def __init__(self, eps=0.001, **kwargs):
        super().__init__(**kwargs)
        self.eps=eps

    def build(self, batch_input_shape):
        self.alpha = self.add_weights(
            name="alpha", shape=batch_input_shape[-1:],
            initializer="zeros")
        self.beta = self.add_weights(
            name="beta", shape=batch_input_shape[-1:],
            initializer="zeros")
        super().build(batch_input_shape) 

    def call(self, X):
        mean, variance = tf.nn.moments(X, axes=1, keepdims=True)
        return self.alpha * (X - mean) / (tf.sqrt(variance + self.eps)) + self.beta

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "eps": self.eps}

Ensure that your custom layer produces the same (or very nearly the same) output as the keras.layers.LayerNormalization layer. Using the california housing database

In [12]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [13]:
x = X_train.astype(np.float32)

In [14]:
myNormLay = myNormalizingLayer()
kerasNormLay = keras.layers.LayerNormalization()

In [23]:
#- ISSUE - May have to deal with windows install of cuDNN? Custom normalization layer isn't playing nicely with cuDNN. Switch to Docker to see if the problem persists
tf.reduce_mean(keras.losses.mean_absolute_error(kerasNormLay(x), myNormLay(x)))

AttributeError: 'myNormalizingLayer' object has no attribute 'add_weights'

In [24]:
random_alpha = np.random.rand(x.shape[-1])
random_beta = np.random.rand(x.shape[-1])

myNormLay.set_weights([random_alpha, random_beta])
kerasNormLay.set_weights([random_alpha, random_beta])

tf.reduce_mean(keras.losses.mean_absolute_error(
    kerasNormLay(x), myNormLay(x)))

ValueError: You called `set_weights(weights)` on layer "my_normalizing_layer_1" with a weight list of length 2, but the layer was expecting 0 weights. Provided weights: [array([0.65162788, 0.1292612 , 0.10223085, 0.4594...